### Install Dependencies

In [1]:
#%pip install nltk

### Imports

In [2]:
import nltk
nltk.download('punkt')
import os
import zipfile
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


### Download NLTK Packages

In [3]:
if not os.path.exists('./packages'):
    os.makedirs('./packages')
nltk.data.path.append('./packages')
nltk.download('all', download_dir='./packages')
with zipfile.ZipFile('./packages/corpora/wordnet.zip', 'r') as zip_ref:
    zip_ref.extractall('./packages/corpora')

[nltk_data] Error loading all: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


## Import Data

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./../../data/intents_generated.json').read()
intents = json.loads(data_file)

## Check Data

In [5]:
intents.keys()

dict_keys(['intents'])

In [6]:
intents['intents'][:5]

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'Come back soon'],
  'context_set': ''},
 {'tag': 'creator',
  'patterns': ['what is the name of your developers',
   'what is the name of your creators',
   'what is the name of the developers',
   'what is the name of the creators',
   'who created you',
   'your developers',
   'your creators',
   'who are your developers',
   'developers',
   'you are made by',
   'you are made by whom',
   'who

##  Data pre-processing

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        # add to classes
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### Lemmatization

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)


474 documents
53 classes ['accommodation_options', 'admission', 'admission_deadlines', 'admission_decisions', 'admission_requirements', 'alumni_success_stories', 'application_process', 'application_submission', 'campus_facilities', 'canteen', 'college intake', 'committee', 'computerhod', 'course', 'creator', 'document', 'eligibility', 'event', 'extchod', 'extracurricular_activities', 'facilities', 'fees', 'financial_aid', 'financial_aid_options', 'floors', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'internship_opportunities', 'ithod', 'library', 'location', 'menu', 'name', 'number', 'placement', 'principal', 'ragging', 'random', 'research_opportunities', 'salutaion', 'scholarship', 'sem', 'sports', 'student_support_services', 'swear', 'syllabus', 'task', 'uniform', 'vacation']
376 unique lemmatized words ["'s", '(', ')', '.', 'a', 'about', 'ac', 'academic', 'accepted', 'accommodation', 'achievement', 'active', 'activity', 'address', 'admision', 'admission', 'aft

a pickle file is made that will be used for prediction.

In [9]:
pickle.dump(words,open('./bin/words.pkl','wb'))
pickle.dump(classes,open('./bin/classes.pkl','wb'))

# Create training and testing data
Now that the training data has been created, the input and output will be included. The pattern will serve as our input, and the class to which the pattern belongs will serve as our output. However, since the machine cannot interpret language, we must convert it to numbers.

In [10]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
train_x = np.array(list(training[:,0]), dtype=np.float32)
train_y = np.array(list(training[:,1]), dtype=np.float32)
print("Training data created")

Training data created


# Build the model

In [11]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
#decay was depreciated thus changes made according to latest version

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Learning rate schedule
initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

# Using Adam optimizer with learning rate schedule
adam = Adam(learning_rate=lr_schedule)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Fitting the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
95/95 [==============================] - 1s 3ms/step - loss: 3.7560 - accuracy: 0.1034
Epoch 2/200
95/95 [==============================] - 0s 3ms/step - loss: 3.0861 - accuracy: 0.2405
Epoch 3/200
95/95 [==============================] - 0s 3ms/step - loss: 2.6580 - accuracy: 0.3333
Epoch 4/200
95/95 [==============================] - 0s 2ms/step - loss: 2.3649 - accuracy: 0.4051
Epoch 5/200
95/95 [==============================] - 0s 2ms/step - loss: 2.3250 - accuracy: 0.4156
Epoch 6/200
95/95 [==============================] - 0s 2ms/step - loss: 2.2768 - accuracy: 0.4430
Epoch 7/200
95/95 [==============================] - 0s 2ms/step - loss: 2.0743 - accuracy: 0.4873
Epoch 8/200
95/95 [==============================] - 0s 2ms/step - loss: 1.9527 - accuracy: 0.5021
Epoch 9/200
95/95 [==============================] - 0s 2ms/step - loss: 1.9370 - accuracy: 0.5127
Epoch 10/200
95/95 [==============================] - 0s 2ms/step - loss: 1.9469 - accuracy: 0.5232
Epoch 11/

## Compile & Save

In [12]:
model.save('./bin/model.h5', hist)
print("model created")

model created


## Test

In [13]:
model = load_model('./bin/model.h5')
intents = json.loads(open('./../../data/intents_hybrid.json').read())
words = pickle.load(open('./bin/words.pkl','rb'))
classes = pickle.load(open('./bin/classes.pkl','rb'))

def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag(sentence, words, show_details=True):
    sentence_words = clean_sentence(sentence)

    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def classify(sentence, model):
    p = bag(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]

    ERROR_THRESHOLD = 0.1

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']

    list_of_intents = intents_json['intents']

    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = classify(text, model)
    res = getResponse(ints, intents)
    return res

In [14]:
chatbot_response('hola')

1/1 [==============================] - 0s 144ms/step


'Hi there, how can I help?'

## Benchmarks

In [15]:

data_file = open('./../../data/benchmarkData.json').read()
seed = json.loads(data_file)

In [16]:
model = load_model('./bin/model.h5')

In [17]:
seed

{'greeting': ['Hello!', 'Hey there!', "Hi, how's it going?"],
 'goodbye': ['Goodbye!', 'See you later!', 'Farewell!'],
 'creator': ['Who created you?',
  'Tell me about your creator.',
  'Who made this bot?'],
 'name': ["What's your university's name?",
  'Tell me the name of your college.',
  'University name?'],
 'hours': ['What are your working hours?',
  'When are you open?',
  'Office hours?'],
 'number': ['Can I get the contact number?',
  'Give me the phone number.',
  'Contact details?'],
 'course': ['Tell me about engineering courses.',
  'Information on computer science program?',
  'Engineering details.'],
 'fees': ['How much are the fees?',
  'Tell me about the costs.',
  'Fees for engineering program?'],
 'location': ['Where is your university located?',
  'Give me the location.',
  'University address?'],
 'hostel': ['Tell me about hostel facilities.',
  'Information on accommodation?',
  'Hostel details.'],
 'event': ['Are there any upcoming events?',
  'Tell me about un

In [18]:
def classify(sentence, model):
    sentence_words = clean_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
    p = np.array([bag])
    res = model.predict(p)[0]
    ERROR_THRESHOLD = 0.10
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    try:
        if return_list[0]:
            return return_list[0]['intent']
    except IndexError:
        pass
    return 'none'

In [19]:
def run_benchmark(model, seed):
    correct = 0
    total = 0

    for class_name, sentences in seed.items():
        for sentence in sentences:
            prediction = classify(sentence, model)
            if prediction == class_name:
                correct += 1
            else:
                print('incorrect classification. expected: '+class_name+', got: '+prediction)
            total += 1

    print(f"Accuracy: {correct / total}")

In [20]:
run_benchmark(model, seed)

1/1 [==============================] - 0s 44ms/step
incorrect classification. expected: goodbye, got: greeting
1/1 [==============================] - 0s 38ms/step
incorrect classification. expected: name, got: none
1/1 [==============================] - 0s 26ms/step
incorrect classification. expected: course, got: computerhod
1/1 [==============================] - 0s 39ms/step
incorrect classification. expected: course, got: none
1/1 [==============================] - 0s 25ms/step
incorrect classification. expected: fees, got: internship_opportunities
1/1 [==============================] - 0s 37ms/step
incorrect classification. expected: location, got: none
1/1 [==============================] - 0s 49ms/step
incorrect classification. expected: hostel, got: canteen
1/1 [==============================] - 0s 36ms/step
incorrect classification. expected: hostel, got: none
1/1 [==============================] - 0s 32ms/step
incorrect classification. expected: document, got: greeting
1/1 [==